In [57]:
import os 
import copy 
import numpy as np
np.set_printoptions(suppress=True, precision=4)

from scipy.spatial.transform import Rotation as R

import open3d as o3d

import symforce 
# symforce.set_log_level("warning")
symforce.set_log_level("ERROR")
print(f"symforce uses {symforce.get_symbolic_api()} as backend")

from symforce.notebook_util import display
import symforce.symbolic as sf
from symforce.values import Values
from symforce import ops
from symforce.ops import StorageOps, GroupOps, LieGroupOps

import symforce.opt.noise_models as nm

symforce uses symengine as backend


In [58]:
from time import time
  
# ref: https://www.geeksforgeeks.org/timing-functions-with-decorators-python/
disp_timecost = True 
def timer(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        
        if disp_timecost:
            print(f'Function {func.__name__} executed in {(t2-t1):.4f}s')

        return result
    return wrap_func

def np2o3d(nx3mat):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(nx3mat)
    return pcd

def to_o3dlineset(points, corres_idxes):
    if len(points) == 0:
        return None
    
    return o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.array(points)),
        lines=o3d.utility.Vector2iVector(np.array(corres_idxes)),
    )

def probabilistic_false_corres_idx(init_idx, num_max, outlier_ratio=0.5):
    if np.random.rand(1) > outlier_ratio:
        # return ture_correspondenceness, corres_idx
        return init_idx, True
    else:
        return int(np.random.randint(num_max, size=(1)).squeeze()), False
#         return 1, False


In [59]:

# Model parameters (as symbolic)
scale    = sf.V1.symbolic("s")
transvec = sf.V3.symbolic("t")
rotvec   = sf.V3.symbolic("Theta") # i.e., angle-axis parametrization
rotmat   = LieGroupOps.from_tangent(sf.Rot3, rotvec) # for debug, display(rotmat.to_rotation_matrix())

# Redisual (loss function)
#  note: the rotation 'matrix' is used to formulate the below constraint, 
#        but it was parametrized as a 3-dim vector 'rotvec'!
p_src        = sf.V3.symbolic("p_src")     # p means a single 3D point 
p_tgt        = sf.V3.symbolic("p_tgt") 

# p_tgt_est    = (rotmat * p_src)*scale + transvec # bug. sometimes converges but sometimes weired behaviors 
p_tgt_est    = ((rotmat * p_src) + transvec)*scale # The constraint: see the eq149 of https://ethaneade.com/lie.pdf
    # for the Sim(3) details, see
        # Scale Drift-Aware Large Scale Monocular SLAM (RSS 2020)
    # and see the eq149 of https://ethaneade.com/lie.pdf

error_val = p_tgt - p_tgt_est
print(f"error_val type is {type(error_val)}")

def robust_loss(error_V3: sf.V3):
    """
    see the class BarronNoiseModel(ScalarNoiseModel) definition in noise_models.py
        alpha: Controls shape and convexity of the loss function. Notable values:
            alpha = 2 -> L2 loss
            alpha = 1 -> Pseudo-huber loss
            alpha = 0 -> Cauchy loss
            alpha = -2 -> Geman-McClure loss
            alpha = -inf -> Welsch loss
        delta: Determines the transition point from quadratic to robust. Similar to "delta" as used
            by the pseudo-huber loss function.
        scalar_information: Scalar representing the inverse of the variance of an element of the
            unwhitened residual. Conceptually, we use "scalar_information" to whiten (in a
            probabalistic sense) the unwhitened residual before passing it through the Barron loss.
        x_epsilon: Small value used for handling the singularity at x == 0.
        alpha_epsilon: Small value used for handling singularities around alpha.
    """

    alpha = 0
    delta = 0.2
    scalar_information = 5.0
    epsilon = 1.0e-6

    noise_model = nm.BarronNoiseModel(
        alpha=alpha, delta=delta, scalar_information=scalar_information, x_epsilon=epsilon
    )

    robustified_error = sf.V1(noise_model.error(error_V3)) # robust loss 

#     robustified_error = error_V3.compute_AtA() #non robust loss 
    
    return robustified_error

error_model = robust_loss(error_val) 

# residual jacobian
#  this is the powerful moment of symforce. It automatically generate the Jacobian equations explicitly. 
Je_trans_model = error_model.jacobian(transvec)
Je_rot_model = error_model.jacobian(rotvec)
Je_scale_model = error_model.jacobian(scale)

# residual debug 
is_vis_jacobians = False

def disp_info(elm, name=''):
    print("=========INFO==========")
    print(f"The shape and equation of {name}:")
    display(elm.shape)
    display(elm)
    print("=======================\n")

if is_vis_jacobians:
    disp_info(error_model, 'error_model')
    disp_info(Je_rot_model, 'Je_rot')
    disp_info(Je_trans_model, 'Je_trans')
    disp_info(Je_scale_model, 'scale')


error_val type is <class 'symforce.geo.matrix.Matrix31'>


In [75]:

# Sim(3) optimization state dimension 
ndim_state = 7
ndim_loss = 1

# The nonlinear icp alg. 
def evaluate_error_and_jacobian(src_pt: np.ndarray, tag_pt: np.ndarray, tf):
    # note: transformation is 6dim vector on the tangent space (i.e., [rotvec, trans])  == lie algebra, aka se(3) (note that "small" se)
    se3 = tf[:6] # [rotvec3dim, trans3dim]
    s = tf[-1] # scale 
    
    def inject_values(model):
        model_evaluated = \
            model.subs({rotvec: sf.V3(se3[:3]), \
                        transvec: sf.V3(se3[3:]), \
                        scale: sf.V1(s), \
                        p_src: sf.V3(src_pt), \
                        p_tgt: sf.V3(tag_pt)})
        return model_evaluated.to_numpy()
        
    error, Je_rot, Je_trans, Je_scale = \
        [inject_values(x) for x in [error_model, Je_rot_model, Je_trans_model, Je_scale_model]]

    return error, Je_rot, Je_trans, Je_scale
    
@timer
def icp_once(src, tgt, tf_init, skip=100, outlier_ratio=0.5, num_iters=30, fix_scale=False, verbose=False):

    for _iter in range(num_iters):
        num_pts = src.shape[0]

        correct_corres_points = []
        correct_corres_indexes = []
        false_corres_points = []
        false_corres_indexes = []

        H = np.zeros((ndim_state, ndim_state))
        b = np.zeros((ndim_state, 1))

        # 1. gathering measurements
        #  these should be parallelized with only locking the H++ and b++ block. C++ would be a choice for this job.
        for pt_idx in range(num_pts):

            # if "true" correspondence is given (this is an tutorial for education purpose), 
            # using a few points okay ..
            if pt_idx % (skip+_iter) != 0:
                continue # to save time cost, ealry return

            # Here, we directly use the true-known pair (because this is a tutorial for educational purpose :)
            #  In practice, (src_pt, tgt_pt) should be a correspondence (e.g., found by FPFH local featuer, see https://pcl.readthedocs.io/projects/tutorials/en/latest/fpfh_estimation.html)
            src_corres_idx = pt_idx 
            tgt_corres_idx, is_true_corres \
                = probabilistic_false_corres_idx(pt_idx, num_pts-1, outlier_ratio=outlier_ratio) # true 

            src_pt, tgt_pt = src[src_corres_idx, :], tgt[tgt_corres_idx, :]
            
            e, Je_rot, Je_trans, Je_scale \
                = evaluate_error_and_jacobian(src_pt, tgt_pt, tf_init)
                #   ps. To understand the details of this nonlinear iterative update steps, see http://www.diag.uniroma1.it//~labrococo/tutorial_icra_2016/icra16_slam_tutorial_grisetti.pdf 
                #       however, in the above slide's example, the jacobian was generated by hand as well as Euler angle space was used, not angle-axis.

            if fix_scale:
                e[-1] = 0.000001
                Je_scale = np.array([[1]])

            J = np.hstack((Je_rot, Je_trans, Je_scale)) 
                # this is 1x7 
                #   1 is observation error model's output dimension 
                #      (in this tutorial, the error model is a norm of 3-dim error-state vector)
                #   7 is the state dimension

            sqrtW = np.array([[1., 1., 1., 1., 1., 1., 1.]])
            J = sqrtW * J # whitened J. (element-wise multiplication)
                            # i.e., J.t@W@J == (sqrtW@J).t @ (sqrtW@J) == ||sqrtW@J||2
                          # In this case, make lower the sensitivity of the scale term will be benefit for the convergnece. 
                          #  This is an engineering. you should apply your physical prior or domain knowledge, or even empirically find it.
            e[:3] *= 0.5 # this is also empirically important. 
            e[-1] *= 0.15 # this is also empirically important. 
                          # because translation changes along a few meters, but rotation and scales are lives in [0, 1]
            
            H = H + (J.T @ J) # H: 7x1 * 1x7 => thus H is 7x7
            b = b + (J.T @ e) # b: 7x1 * 1x1 => thus b is 7x1
                # to understatand the update eq, see https://darkpgmr.tistory.com/142

            if verbose:
                print("\n=================")
                print(f"{pt_idx} error is\n{e.T}")
                print(f"{pt_idx} Je_rot is\n{Je_rot}")
                print(f"{pt_idx} Je_trans is\n{Je_trans}")
                print(f"{pt_idx} J is\n{J}")
                print(f"{pt_idx} H is\n{H}")
                print(f"{pt_idx} b is\n{b}")

            # debug 
            if is_true_corres:
                correct_corres_points.append(src_pt)
                correct_corres_points.append(tgt_pt)
                correct_corres_indexes.append([len(correct_corres_indexes)*2, len(correct_corres_indexes)*2+1])
            else:
                false_corres_points.append(src_pt)
                false_corres_points.append(tgt_pt)
                false_corres_indexes.append([len(false_corres_indexes)*2, len(false_corres_indexes)*2+1])


        # 2. update once 
        dtf = -np.linalg.solve(H, b).squeeze() # note the step direction is minus

        
        # debug
        correct_corres_line_set = to_o3dlineset(correct_corres_points, correct_corres_indexes)
        false_corres_line_set = to_o3dlineset(false_corres_points, false_corres_indexes)
        line_sets = {"correct": correct_corres_line_set, 
                     "false": false_corres_line_set}

        # update 
        strange_update_alram_thres = 100.0
        if np.linalg.norm(dtf) > strange_update_alram_thres:
            # strange_update_alram_thres is just arbitrarily selected because this is a toy problem 
            # if a weired update is detected, do not apply it.  
            print(f"dtf norm: {np.linalg.norm(dtf):.3f} is weired. Thus reject to update.")
            break

        tf_init = tf_init + dtf # updated within the tangent space
        print(f"the estimated relative tf for iter {_iter} is {tf_init}")

    # final result 
    tf = tf_init

    return tf, line_sets


In [76]:
# Data generatation  
#  source 
dataset_name = "dragon"
pcd0 = o3d.io.read_point_cloud(f'data/{dataset_name}.pcd')

scale_up = 10
pcd0_points_scaled_up = np.array(pcd0.points) * scale_up
pcd0 = np2o3d(pcd0_points_scaled_up)
print(pcd0)
print(f" The datset metric scale min {np.min(np.array(pcd0.points), 0)}")
print(f" The datset metric scale max {np.max(np.array(pcd0.points), 0)}")

#  generate target 
def rpy2mat(rpy, deg=True):
    return R.from_euler('xyz', rpy, degrees=deg).as_matrix()

def rpy2vec(rpy, deg=True):
    return R.from_euler('xyz', rpy, degrees=deg).as_rotvec()

true_rot_diff_rpy = np.array([0, 0, 115]) # deg 
true_rot_diff = rpy2mat(true_rot_diff_rpy)
true_rot_diff_vec = rpy2vec(true_rot_diff_rpy)
true_trans_diff = np.array([-0.1335, 0.15, 0.05]) * (0.1*scale_up)
true_scale_diff = 5.0

print(f"\ntrue_rot_diff is\n {true_rot_diff}")
print(f"true_rot_diff_vec is\n {true_rot_diff_vec}")
print(f"true_trans_diff is\n {true_trans_diff}")
print(f"true_scale_diff is\n {true_scale_diff}")

pcd1 = o3d.geometry.PointCloud()
pcd1_Sim3_applied = true_scale_diff*(true_rot_diff @ np.array(pcd0.points).transpose()) + np.expand_dims(true_trans_diff, axis=-1)
pcd1.points = o3d.utility.Vector3dVector(pcd1_Sim3_applied.transpose())
    

PointCloud with 5205 points.
 The datset metric scale min [-1.0758  0.5284 -0.4984]
 The datset metric scale max [0.9524 1.9634 0.4083]

true_rot_diff is
 [[-0.4226 -0.9063  0.    ]
 [ 0.9063 -0.4226  0.    ]
 [ 0.      0.      1.    ]]
true_rot_diff_vec is
 [0.     0.     2.0071]
true_trans_diff is
 [-0.1335  0.15    0.05  ]
true_scale_diff is
 5.0


In [ ]:
##########
#  MAIN 
##########

# At the very first status 
is_viz = 1
if is_viz:
    pcd0.paint_uniform_color([1, 0, 1])
    pcd1.paint_uniform_color([0, 0, 1])
    o3d.visualization.draw_geometries([pcd0, pcd1], window_name="initial status")

# Initial condition 
def gen_noisy_but_reliable_inital():
    rot_init = R.from_euler('xyz', true_rot_diff_rpy, degrees=True).as_rotvec() + 0.05*np.random.rand(3)
    trans_init = true_trans_diff + np.random.rand(3)*(0.1*scale_up)
    scale_init = 0.7* true_scale_diff
    
    eps = 0.0001 # +eps means: because zero initial should be avoided (see the symbolic equation of Je_rot!)
    initial_state_vector = np.hstack((rot_init, trans_init, scale_init)) + eps 
    return initial_state_vector

def identity_inital():
    eps = 0.01
    return np.array([eps, eps, eps, eps, eps, eps, 1.0])
    # because after the update, the registered_src is expected to be equal to the target 
    # thus, the translation and rotataion would be zero and the relative scale must be 1.0

init_guess = gen_noisy_but_reliable_inital() 
# init_guess = gen_noisy_but_reliable_inital() 
    # at the very first step, a moderate (i.e., not-identity) initial value is required 
    # because the cost function is highly nonlinear
    # ps. try yourself using init_guess = identity_inital() rather than gen_noisy_but_reliable_inital()
    #  the convergence speed would be deteriorated. (test yourself!)
print(f"init_guess is {init_guess}")

# NOTE
# The number of correspondences and their spatial distirbution would affect the results
# for example, 
# in the below example, 
# for the dragon dataset, it will converge (when we use the robust loss) even under 50% outliers while using skip=20
# however, for the bunny dataset, which has the more smaller number of points, would not converge when we use skip=20 (skip=1 is then okay. try yourself!)
    # ps. for the production level code, you also adaptively conclude when num_iters should be stopped in the icp_once (e.g., by tracking the df or residuals)
        # by doing so, you need to prevent the solution from divergent.   
# therefore, the what I want to say is for parameter tuning, we should well understand your dataset's characteristics (e.g., density, spatial distribution, etc.)

# the robust loss parameter is also important
# for dragon, we will use 
"""
    alpha = 0
    delta = 0.1
    scalar_information = 10
    epsilon = 1.0e-6
    
    , and pts_skip = 20
"""

# ICP starts  
max_iter = 25
src_pc, tgt_pc = [np.array(pc.points) for pc in [pcd0, pcd1]]
for _iter in range(max_iter):
    print(f"\n======================================")
    print(f"  ==========   iter {_iter}   ==========")
    print(f"======================================")
    # 1. optimize once 
    src_pc_before_updated = copy.deepcopy(src_pc)

    outlier_ratio = 0.5 # test yourself up to 0.00 (no outlier) to 0.99
    pts_skip = (10 + _iter) # for bunny (num points are small), use skip = 1 and for the dragon, use skip=20 is okay
    num_iters = 20

    if _iter < 0:
        fix_scale=True 
    else:
        fix_scale=False
    tf_tangent, line_sets = icp_once(src_pc, tgt_pc, init_guess, skip=pts_skip, \
                                     outlier_ratio=outlier_ratio,
                                     num_iters=num_iters,
                                     fix_scale=fix_scale,
                                     verbose=False) # if "true" correspondence is given (this is an tutorial for education purpose), using a few iteration okay ..

    # 2. move the src to target and 
    est_rot3, est_trans3, est_scale = tf_tangent[:3], tf_tangent[3:6], tf_tangent[-1]
    est_rotmat3x3 = rotmat.subs({rotvec: sf.V3(est_rot3)}).to_rotation_matrix().to_numpy()
    src_pc_updated = est_scale*(est_rotmat3x3 @ src_pc.transpose()) + np.array([est_trans3]).transpose()
    src_pc = src_pc_updated.transpose()
    
    init_guess = identity_inital()
    # note: we explicitly update the source point cloud (i.e., registered), 
        # thus from the next step, we will use init_guess always equal to identity (toy example assumption)
        # because, as already mentioned, after the update, the registered_src is expected to be equal to the target 
        # thus, the translation and rotataion would be zero and the relative scale must be 1.0
        # in real world example, we need to use a domain knowledge to update the better initial (e.g., constant motion model, the prior knowledge of the object's metric scale, etc.)

    # 3. re-correspondence
    #  here, we can use the known true-correspondence because this is just a tutorial and affine transformation does not change the true correspondences 
    #   but in real world applications, kd-tree-like nearest neighbor search to find a newaly updated correspondences is required. 

    # 4. debug: Verify the result visually 
    if is_viz:
        pcd0_Sim3_before_update = np2o3d(src_pc_before_updated)
        pcd0_Sim3_before_update.paint_uniform_color([0.5, 0.5, 0.5])
        
        pcd0_Sim3_after_update = np2o3d(src_pc)
        pcd0_Sim3_after_update.paint_uniform_color([25./255, 158./255, 243./255])
        
        pcd1.paint_uniform_color([0, 0, 1])
        
        line_sets["correct"].paint_uniform_color([0, 0.737, 0.354])
        if line_sets["false"] is None:
            line_sets["false"] = copy.deepcopy(line_sets["correct"])
        else:
            line_sets["false"].paint_uniform_color([1.0, 0.0, 0.0])
        
        # draw before 
        o3d.visualization.draw_geometries([pcd0_Sim3_before_update, pcd1, \
                                           line_sets["correct"], line_sets["false"]], \
                                           window_name=f"iteration {_iter} (gray: before update, blue: target)")
        
        # draw after 
        o3d.visualization.draw_geometries([pcd0_Sim3_after_update, pcd1], \
                                           window_name=f"iteration {_iter} (sky: after updated, blue: target)")

    # 5. if tf_tangent is smaller than a threshold, stop 
    print("\n==========estimation==========")
    print(f"delta rotation:\n{est_rotmat3x3}")
    print(f"delta translation:\n{est_trans3}")
    print(f"delta scale: {est_scale:.3f}")
    # TBA, e.g., if delta_translation < 0.01, break;


init_guess is [0.0243 0.0285 2.0356 0.8222 0.4198 0.7619 3.5001]

  ==========   iter 0   ==========
the estimated relative tf for iter 0 is [-0.1013  0.0273  2.0071  0.7276  0.3442  0.2578  3.2702]
the estimated relative tf for iter 1 is [-0.0766 -0.0472  1.9578  0.4733  0.2325  0.2494  3.0476]
the estimated relative tf for iter 2 is [-0.0827 -0.0884  2.015   0.2863  0.2181  0.227   3.0278]
the estimated relative tf for iter 3 is [-0.1134 -0.118   2.0236  0.1062  0.1933  0.2217  3.0253]
the estimated relative tf for iter 4 is [-0.1037 -0.084   2.011   0.0479  0.1436  0.1579  3.0873]
the estimated relative tf for iter 5 is [-0.0643 -0.064   2.0104  0.0061  0.1103  0.1019  3.228 ]
the estimated relative tf for iter 6 is [-0.0698 -0.0726  2.0462 -0.0945  0.1265  0.0983  3.3513]
the estimated relative tf for iter 7 is [-0.0544 -0.0564  2.0446 -0.1987  0.1112  0.073   3.3385]
the estimated relative tf for iter 8 is [-0.0655 -0.0411  2.0516 -0.2386  0.1043  0.0591  3.3751]
the estimated rel

the estimated relative tf for iter 12 is [-0.0085  0.0182  0.0013 -0.0601  0.0433 -0.0883  0.9888]


In [ ]:
# @ Future works 

# 1: 
# Here, the false residual and its Hessian is directly incorporated within the normal equation 
# we can say this deweighting method implictily handles the outliers. 
# The next step you can do is to "explictly remove" the false correspondences from the pairs 
# e.g., using RANSAC 

# 2:
# using parallel implmentation, boost the above per-point iterations faster 

# 3:
# using not a point-to-point L2 distance, using point-to-plane (using normal and dot product) loss

